DIP-STER Tutorial

The purpose of this section is to show how to utilize the DIP-STER library

## Table of Contents
- [Data Preperation](#11-data-preparation)
- [Network Setup](#12-network-setup)
- [Outputting and Saving Data](#13-outputting-and-saving-data)

In [ ]:
import dipster as dip
from dipster import DeviceContextEnum, DtypesEnum
import tomobase
import tomondt
import torch

## 1.1. Data Preparation
Data imports, exports, and processing tasks are handled by the [Time Dependent Tomography](https://github.com/Tcraig088/TimeDependentTomography) library. Please see library for details. 

For setting up a DIP-STER experiment the tilt series / sinogram is required. This can be imported using the tomobase submodule of the Time Dependent Tomography library.  

Sometimes it is useful to monitor the training process by default the SSIM and loss are monitored every 100 cycles. This is achieved using [WandB](https://wandb.ai/site/). If a volume-time series is avalaible, orthoslices can be visually compared to the volume time series. Volume time series can be imported using the tomobase submodule of the Time Dependent Tomography library.

Its worth noting that the data must be reformatted from the default of the Time Dependent Tomography Library. By convention, the Time Dependent Tomography library utilize an image format of (n, x, z) whereas DIP-STER is (z, x, n). The convert function will by default correct this and convert the dataset to use Pytorch which is unsupported in the data processing library. 

Ensure all image dimensions (except number of projections) is a multiple of 2. By default the network supports images of 128 pixels but can support any dimensions fulfilling this requirement with optimization. 

In [ ]:
# Assume sinogram and volume time series are imported as nessessary

sino = dip.convert(sino)
sino.setcontext(DeviceContextEnum.TORCH, DtypesEnum.FLOAT32)

#volseries = dip.convert(volseries)
#volseries.setcontext(DeviceContextEnum.TORCH, DtypesEnum.FLOAT32)


## 1.2. Network Setup
To set up the network, initialize the network, set its parameters and train by inputting the sinogram. All network paramaters can be found in params.py. 

running the eval() command will allow intermediary training steps to be seen in wandb. If run with a volume series the comparitive metrics will be calculated from the volume series compared to the orthoslice output. By default the evaluation is performed between the tiltseries and the forward projected output. Note: loss function is always calculated from the tilt series.  



In [ ]:

net = dip.Solver(sino)

net.params.wandb_project = 'TestProject'
meta = net.params.to_dict()

net.eval()
#net.eval(volseries)
net.train(sino)

# 1.3. Outputting and Saving Data
The reconstructed volume can be obtained for a reconstruction time by inputting the desired reconstruction times into the network. The network supports being saved or loaded as a pkl file.

Note: the Time Dependent Tomography library supports lazy loading of compressed volume time series files into Napari for visualization. These files can be generated by adding the full volume time series into a file using the write_record function

In [ ]:
# get volume 
for volume, time in net.reconstruct(times=sino.times):
    # do what you want with the volume
    pass

In [ ]:
# save to pkl file
torch.save(net.state_dict(), 'filename.pkl')

# load from pkl file
net = dip.Solver.from_state_dict(torch.load('filename.pkl'))

In [ ]:

# Convert to VMF file for Napari visualization
result = tomondt.new_vmf('filename.vmf')
for variables in net.reconstruct(times=sino.times):
    result.write_record(*variables)